# Automation Excel to GeoJSON and Shapefile

## 1. Application to Export Excel into GeoJson

### 1.1. Function Codes

In [10]:
import sys
import subprocess
import pkg_resources
import json
import glob
import os
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiPoint

def install_requirements(): #Install required packages if they are not already installed.
    required = {
        'pandas': '1.0.0',
        'openpyxl': '3.0.0',
        'geopandas': '0.9.0',
        'shapely': '1.7.0',
    }
    
    # Check what's installed
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    
    # Determine what needs to be installed
    missing = []
    update = []
    
    for package, min_version in required.items():
        if package not in installed:
            missing.append(package)
        elif pkg_resources.parse_version(installed[package]) < pkg_resources.parse_version(min_version):
            update.append(package)
    
    # If packages need to be installed or updated
    if missing or update:
        print("Some required packages are missing or need to be updated.")
        print(f"Missing: {', '.join(missing) if missing else 'None'}")
        print(f"Need update: {', '.join(update) if update else 'None'}")
        
        try:
            # Install missing packages
            if missing:
                print(f"Installing missing packages: {', '.join(missing)}")
                subprocess.check_call([sys.executable, "-m", "pip", "install"] + missing)
            
            # Update packages that need updating
            if update:
                print(f"Updating packages: {', '.join(update)}")
                subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade"] + update)
                
            print("All required packages have been installed/updated successfully!")
            
            # Re-import the modules to make them available
            if 'pandas' in missing or 'pandas' in update:
                global pd
                import pandas as pd
            if 'openpyxl' in missing or 'openpyxl' in update:
                global load_workbook
                from openpyxl import load_workbook
            if 'geopandas' in missing or 'geopandas' in update:
                global gpd
                import geopandas as gpd
            if 'shapely' in missing or 'shapely.geometry' in update:
                global Point, LineString
                from shapely.geometry import Point, LineString
                
        except Exception as e:
            print(f"Failed to install required packages: {str(e)}")
            print("Please manually install the required packages using:")
            print("pip install pandas openpyxl geopandas shapely")
            sys.exit(1)


install_requirements()


from shapely.geometry import Point, LineString, MultiPoint

def unique_column_names(columns): # Ensure column names are unique by appending suffix.
    seen = {}
    new_columns = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)
    return new_columns

def clean_column_names(columns): # Standardize column names by capitalizing each word properly.
    cleaned_columns = []
    seen = {}
    for col in columns:
        col = str(col).strip()
        
        # Remove columns with "Rekap" in the name
        if "rekap" in col.lower():
            continue
        
        # Fix for duplicated words (like "No No", "Detail Lokasi Detail Lokasi")
        words = col.split()
        if len(words) >= 2:
            # Check for repeated word patterns
            half_length = len(words) // 2
            if words[:half_length] == words[half_length:] and len(words) % 2 == 0:
                # If the first half matches the second half, only use the first half
                col = " ".join(words[:half_length])
        
        # Capitalize words
        col = " ".join(word.capitalize() for word in col.split())
        
        # Handle duplicates
        if col in seen:
            seen[col] += 1
            col = f"{col} {seen[col]}"
        else:
            seen[col] = 0
        
        cleaned_columns.append(col)
    return cleaned_columns

def fix_coordinates(row, lat_col, lon_col): #Fix latitude and longitude values that may be in the wrong format.
    lat, lon = row[lat_col], row[lon_col]
    original_lat, original_lon = lat, lon  # Store original values for debugging
    
    # Handle string values with commas (e.g., "-69,694,951" or "1,065,663,308")
    if isinstance(lat, str):
        try:
            lat = float(lat.replace(',', ''))
        except (ValueError, AttributeError):
            lat = pd.NA
    
    if isinstance(lon, str):
        try:
            lon = float(lon.replace(',', ''))
        except (ValueError, AttributeError):
            lon = pd.NA
    
    # First attempt to detect the scale by number of digits
    if pd.notna(lat):
        lat_abs = abs(lat)
        # For values like -6448977 (which should be around -6.4 degrees)
        if 1_000_000 < lat_abs < 10_000_000 and str(int(lat_abs)).startswith(('6', '7', '8', '9')):
            lat = lat / 1_000_000
        elif lat_abs > 90:
            # General scaling rules based on magnitude
            if lat_abs > 10_000_000:  # Very large values
                lat = lat / 10_000_000
            elif lat_abs > 1_000_000:  # Large values (common for Indonesia coords)
                lat = lat / 1_000_000
            elif lat_abs > 90_000:
                lat = lat / 1_000
    
    if pd.notna(lon):
        lon_abs = abs(lon)
        # For Indonesian longitudes (usually around 106-110 degrees)
        if 100_000_000 < lon_abs < 1_500_000_000:
            lon = lon / 10_000_000
        elif lon_abs > 180:
            # General scaling rules based on magnitude
            if lon_abs > 10_000_000:
                lon = lon / 10_000_000
            elif lon_abs > 1_000_000:
                lon = lon / 1_000_000
            elif lon_abs > 180_000:
                lon = lon / 1_000
    
    # Final validation with better error reporting
    if pd.notna(lat) and (lat < -90 or lat > 90):
        print(f"Warning: Invalid latitude after correction: {lat} (original: {original_lat})")
        
        # Last attempt for specific problematic values
        if -10000 < lat < -90 or 90 < lat < 10000:
            lat = lat / 100
            print(f"  Attempting additional scaling: now {lat}")
        
        if lat < -90 or lat > 90:  # Still invalid
            lat = pd.NA
    
    if pd.notna(lon) and (lon < -180 or lon > 180):
        print(f"Warning: Invalid longitude after correction: {lon} (original: {original_lon})")
        
        # Last attempt for specific problematic values
        if -18000 < lon < -180 or 180 < lon < 18000:
            lon = lon / 100
            print(f"  Attempting additional scaling: now {lon}")
            
        if lon < -180 or lon > 180:  # Still invalid
            lon = pd.NA
        
    return pd.Series([lat, lon])


def clean_geojson(gdf, output_path):  # Save GeoDataFrame in a clean format GeoJSON file.
    temp_path = output_path.replace(".geojson", "_temp.geojson")
    gdf.to_file(temp_path, driver="GeoJSON")

    with open(temp_path, "r", encoding="utf-8") as file:
        geojson_data = json.load(file)

    with open(output_path, "w", encoding="utf-8") as file:
        json.dump(geojson_data, file, indent=4)

    print(f"✅ Saved: {output_path}")

def find_coordinate_columns(df, prefix, column_type): #Find coordinate columns with various naming patterns returns column name if found, None otherwise

    patterns = []
    
    if prefix == 'start':
        if column_type == 'lat':
            patterns = [
                'awal latitude', 'awal lat', 
                'koordinat latitude', 'koordinat lat',
                'latitude awal', 'lat awal',
                'latitude 1', 'lat 1'
            ]
        else:  # longitude
            patterns = [
                'awal longitude', 'awal lon', 
                'koordinat longitude', 'koordinat lon',
                'longitude awal', 'lon awal',
                'longitude 1', 'lon 1'
            ]
    else:  # end
        if column_type == 'lat':
            patterns = [
                'akhir latitude', 'akhir lat',
                'latitude akhir', 'lat akhir',
                'latitude 2', 'lat 2'
            ]
        else:  # longitude
            patterns = [
                'akhir longitude', 'akhir lon',
                'longitude akhir', 'lon akhir',
                'longitude 2', 'lon 2'
            ]
    
    # If we're looking for start coordinates, and just a single coordinate exists (no start/end distinction)
    if prefix == 'start' and column_type == 'lat':
        # Add cases where latitude exists without start/end specifier
        patterns.extend(['latitude', 'lat'])
    elif prefix == 'start' and column_type == 'lon':
        # Add cases where longitude exists without start/end specifier
        patterns.extend(['longitude', 'lon'])
    
    # Search for column containing any of the patterns
    for pattern in patterns:
        for col in df.columns:
            if isinstance(col, str) and pattern in col.lower():
                return col
    
    return None

def flatten_excel_to_geojson(file_path, output_folder, workbook_name):  # Modified to accept workbook_name parameter
    # Load workbook
    wb = load_workbook(file_path, data_only=True)

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Unmerge cells and fill values
        for merge in list(ws.merged_cells):
            ws.unmerge_cells(str(merge))
            top_left = ws.cell(merge.min_row, merge.min_col).value
            for row in range(merge.min_row, merge.max_row + 1):
                for col in range(merge.min_col, merge.max_col + 1):
                    ws.cell(row, col, top_left)

        # Convert to DataFrame
        data = list(ws.values)
        df = pd.DataFrame(data)

        # Check if DataFrame is empty or has too few rows
        if df.empty or len(df) < 3:
            print(f"⚠️ Skipping '{sheet_name}' (Empty sheet or insufficient data)")
            continue

        # Find rows containing "NO" (safer approach)
        try:
            header_indices = df[df.apply(lambda x: x.astype(str).str.contains("NO", case=False, na=False)).any(axis=1)].index
            if len(header_indices) == 0:
                print(f"⚠️ Skipping '{sheet_name}' (No header row with 'NO' found)")
                continue
            header_index = header_indices[0]
        except Exception as e:
            print(f"⚠️ Skipping '{sheet_name}' (Error finding header row: {str(e)})")
            continue

        # Use the identified header row
        df.columns = df.iloc[header_index].astype(str).str.strip()

        # Remove empty columns
        df = df.dropna(axis=1, how="all")

        # First pass of REKAP filtering - do this early and more aggressively
        try:
            rekap_mask = df.columns.astype(str).str.contains("REKAP", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} REKAP columns in first pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering REKAP columns - {str(e)}")

        # Drop rows above and including header, plus the empty row after header
        rows_to_drop = list(range(0, header_index + 2))
        if len(df) > max(rows_to_drop) + 1:  # Make sure we have enough rows
            df = df.drop(index=rows_to_drop).reset_index(drop=True)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (Not enough data rows after header)")
            continue

        # Improved header row merging
        if len(df) >= 2:
            # Convert all values to strings to handle None values safely
            first_row = df.iloc[0].astype(str).replace('None', '').replace('nan', '')
            second_row = df.iloc[1].astype(str).replace('None', '').replace('nan', '')
            
            # Smart merging to avoid duplication
            merged_header = []
            for a, b in zip(first_row, second_row):
                a = a.strip()
                b = b.strip()
                
                # Skip columns with "REKAP" in the name - additional check
                if "rekap" in a.lower() or "rekap" in b.lower():
                    merged_header.append("TO_BE_REMOVED")  # Mark for removal
                    continue
                
                if not a and not b:
                    merged_header.append("Column_" + str(len(merged_header)))
                elif not a:
                    merged_header.append(b)
                elif not b:
                    merged_header.append(a)
                else:
                    # Check if one is contained in the other to avoid redundancy
                    if a.lower() in b.lower():
                        merged_header.append(b)
                    elif b.lower() in a.lower():
                        merged_header.append(a)
                    else:
                        merged_header.append(f"{a} {b}")
            
            # Ensure column names are unique
            df.columns = unique_column_names(merged_header)
            
            # Remove any columns marked for removal
            df = df.loc[:, ~df.columns.str.contains("TO_BE_REMOVED")]
            
            # Remove the first two rows used for headers
            df = df.drop(index=[0, 1]).reset_index(drop=True)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (Not enough rows for headers)")
            continue

        # Second pass of REKAP filtering after merging headers
        try:
            rekap_mask = df.columns.astype(str).str.contains("REKAP", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} REKAP columns in second pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering REKAP columns (second pass) - {str(e)}")

        # Normalize column names for consistent detection (safely handle None values)
        df.columns = [str(col).lower().strip() if col is not None else f"col_{i}" 
                      for i, col in enumerate(df.columns)]
        
        # Third pass of REKAP filtering after normalizing
        try:
            rekap_mask = df.columns.str.contains("rekap", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} rekap columns in third pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering rekap columns (third pass) - {str(e)}")
        
        # Check if this sheet is about MARKA or PAGAR PENGAMAN
        is_marka_sheet = "marka" in sheet_name.lower() or any(col for col in df.columns if isinstance(col, str) and "marka" in col.lower())
        is_pagar_pengaman_sheet = "pagar pengaman" in sheet_name.lower() or any(col for col in df.columns if isinstance(col, str) and "pagar pengaman" in col.lower())

        # Find all coordinate columns using the improved function
        start_lat_col = find_coordinate_columns(df, 'start', 'lat')
        start_lon_col = find_coordinate_columns(df, 'start', 'lon')
        end_lat_col = find_coordinate_columns(df, 'end', 'lat')
        end_lon_col = find_coordinate_columns(df, 'end', 'lon')
        
        # Check available coordinate patterns
        has_start_coords = start_lat_col is not None and start_lon_col is not None
        has_end_coords = end_lat_col is not None and end_lon_col is not None

        if not has_start_coords:
            print(f"⚠️ Skipping '{sheet_name}' (No valid start coordinate columns found)")
            continue

        # Convert all coordinate columns to numeric
        if has_start_coords:
            df[start_lat_col] = pd.to_numeric(df[start_lat_col], errors='coerce')
            df[start_lon_col] = pd.to_numeric(df[start_lon_col], errors='coerce')
            
            # Fix coordinates if needed
            df[[start_lat_col, start_lon_col]] = df.apply(fix_coordinates, axis=1, lat_col=start_lat_col, lon_col=start_lon_col)
        
        if has_end_coords:
            df[end_lat_col] = pd.to_numeric(df[end_lat_col], errors='coerce')
            df[end_lon_col] = pd.to_numeric(df[end_lon_col], errors='coerce')
            
            # Fix coordinates if needed
            df[[end_lat_col, end_lon_col]] = df.apply(fix_coordinates, axis=1, lat_col=end_lat_col, lon_col=end_lon_col)
        
        # Check if the end coordinates actually contain valid data
        if has_end_coords:
            has_valid_end_coords = not df[end_lat_col].isna().all() and not df[end_lon_col].isna().all()
            
            # Check if we have at least one row with both coordinates valid
            valid_pairs = ((df[start_lat_col].notna() & df[start_lon_col].notna()) & 
                           (df[end_lat_col].notna() & df[end_lon_col].notna())).any()
        else:
            has_valid_end_coords = False
            valid_pairs = False
        
        # Determine geometry type based on available coordinates, actual data, and sheet type
        if is_marka_sheet:
            # MARKA sheets should use MultiPoint geometry
            print(f"Processing '{sheet_name}' as MultiPoint (MARKA sheet)")
            
            # Create MultiPoint geometry
            df["geometry"] = df.apply(
                lambda row: MultiPoint([
                    (row[start_lon_col], row[start_lat_col]),
                    (row[end_lon_col], row[end_lat_col])
                ]) if (pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) and 
                       pd.notna(row[end_lon_col]) and pd.notna(row[end_lat_col])) else 
                    (MultiPoint([(row[start_lon_col], row[start_lat_col])]) 
                     if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) else None),
                axis=1
            )
            
            # Exclude coordinate columns from properties
            exclude_cols = [start_lat_col, start_lon_col]
            if has_end_coords:
                exclude_cols.extend([end_lat_col, end_lon_col])
            exclude_cols.append("geometry")
            
        elif is_pagar_pengaman_sheet and has_valid_end_coords and valid_pairs:
            # PAGAR PENGAMAN sheets with valid start/end coordinates should use LineString
            print(f"Processing '{sheet_name}' as LineString (PAGAR PENGAMAN sheet)")
            
            # Create LineString geometry
            df["geometry"] = df.apply(
                lambda row: LineString([
                    (row[start_lon_col], row[start_lat_col]),
                    (row[end_lon_col], row[end_lat_col])
                ]) if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) and 
                     pd.notna(row[end_lon_col]) and pd.notna(row[end_lat_col]) else None,
                axis=1
            )
            
            # Exclude coordinate columns from properties
            exclude_cols = [start_lat_col, start_lon_col, end_lat_col, end_lon_col, "geometry"]
            
        else:
            # Other sheets use Point geometry (only start coordinates)
            print(f"Processing '{sheet_name}' as Point geometry (regular sheet)")
            
            # Create Point geometry with start coordinates
            df["geometry"] = df.apply(
                lambda row: Point(row[start_lon_col], row[start_lat_col]) 
                if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) else None,
                axis=1
            )
            
            # Exclude coordinate columns from properties
            exclude_cols = [start_lat_col, start_lon_col]
            if has_end_coords:
                exclude_cols.extend([end_lat_col, end_lon_col])
            exclude_cols.append("geometry")
        
        # Drop rows where geometry is None
        df = df.dropna(subset=["geometry"]).reset_index(drop=True)

        # Only create GeoDataFrame if there are valid geometries
        if len(df) > 0 and not df["geometry"].isnull().all():
            # Fourth pass of REKAP filtering
            # Filter REKAP columns before creating properties
            properties_cols = [col for col in df.columns if col not in exclude_cols]
            properties_cols = [col for col in properties_cols if 'rekap' not in str(col).lower()]
            
            gdf = gpd.GeoDataFrame(df[properties_cols + ["geometry"]], crs="EPSG:4326")
            
            # Apply column renaming after creating the GeoDataFrame
            gdf.columns = clean_column_names(gdf.columns)

            # Fifth pass REKAP filter - case insensitive filter after cleaning column names
            gdf = gdf.loc[:, ~gdf.columns.astype(str).str.contains("rekap", case=False, na=False)]
            
            # Remove unwanted "None_" and "None" columns
            gdf = gdf.loc[:, ~gdf.columns.astype(str).str.match(r"^None$|None_", na=False)]

            # Remove " None" from remaining column names
            gdf.columns = gdf.columns.astype(str).str.replace(r"\sNone\b", "", regex=True).str.strip()

            # Create output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Define output file path with the new naming convention: WorkbookName_SheetName.geojson
            output_path = os.path.join(output_folder, f"{workbook_name}_{sheet_name}.geojson")

            # One final check for REKAP columns before saving
            if any('rekap' in str(col).lower() for col in gdf.columns):
                print(f"Warning: Some REKAP columns still exist after all filtering: {[col for col in gdf.columns if 'rekap' in str(col).lower()]}")
                # Last resort: manually drop these columns
                gdf = gdf.loc[:, ~gdf.columns.astype(str).str.contains("rekap", case=False, na=False)]

            # Save the GeoJSON in a clean format
            clean_geojson(gdf, output_path)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (No valid geometry found)")
            continue  # Skip processing this sheet if no valid geometries exist

def cleanup_temp_files(output_folder): # Delete temporary files
    for temp_file in glob.glob(os.path.join(output_folder, "*_temp.geojson")):
        try:
            os.remove(temp_file)
        except Exception as e:
            print(f"Error removing temporary file {temp_file}: {str(e)}")


def process_excel_folder(input_folder, output_base_folder): #Process all Excel files in a folder and convert them to GeoJSON
    
    # Create the 'Extract GeoJSON' folder inside the output folder
    output_folder = os.path.join(output_base_folder, "Extract GeoJSON")
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all Excel files in the input folder
    excel_extensions = ['*.xlsx', '*.xls', '*.xlsm']
    excel_files = []
    
    for ext in excel_extensions:
        excel_files.extend(glob.glob(os.path.join(input_folder, ext)))
    
    if not excel_files:
        print(f"⚠️ No Excel files found in {input_folder}")
        return
    
    # Process each Excel file
    for i, file_path in enumerate(excel_files, 1):
        file_name = os.path.basename(file_path)
        workbook_name = Path(file_name).stem  # Get the workbook name without extension
        print(f"\n[{i}/{len(excel_files)}] Processing: {file_name}")
        
        try:
            # Pass the output_folder directly and the workbook_name
            flatten_excel_to_geojson(file_path, output_folder, workbook_name)
            print(f"✓ Completed processing: {file_name}")
        except Exception as e:
            print(f"❌ Error processing {file_name}: {str(e)}")
    
    print(f"\n🎉 All Excel files processed. Output saved to: {output_folder}")
    
    # Clean up temporary files
    cleanup_temp_files(output_folder)

### 1.2. Run Function Excel to GeoJSON

In [11]:
input_folder = r"C:\Users\kanzi\Documents\Part Time Job\Data Hasil Survey1"  # Fill with the path file of excel
output_base_folder = r"C:\Users\kanzi\Documents\Part Time Job\Hasil Export"  # Fill with the path folder of export result
process_excel_folder(input_folder, output_base_folder) # Run the function!


[1/31] Processing: 01. Cileungsi - Cibeet.xlsx


c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Cibeet_RAMBU.geojson
Processing 'PJU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Cibeet_PJU.geojson
Processing 'RPPJ' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Cibeet_RPPJ.geojson
Processing 'PAGAR PENGAMAN' as LineString (PAGAR PENGAMAN sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Cibeet_PAGAR PENGAMAN.geojson
Processing 'MARKA' as MultiPoint (MARKA sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Cibeet_MARKA.geojson
Processing 'WARNING LIGHT' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\01. Cileungsi - Ci

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\02. CILEUNGSI - CIBINONG (CITEUREUP)_RAMBU.geojson
Processing 'PJU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\02. CILEUNGSI - CIBINONG (CITEUREUP)_PJU.geojson
Processing 'RPPJ' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\02. CILEUNGSI - CIBINONG (CITEUREUP)_RPPJ.geojson
Processing 'PAGAR PENGAMAN' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\02. CILEUNGSI - CIBINONG (CITEUREUP)_PAGAR PENGAMAN.geojson
Processing 'MARKA' as MultiPoint (MARKA sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\02. CILEUNGSI - CIBINONG (CITEUREUP)_MARKA.geojson
Processing 'WARNING LIGHT' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Docu

...

Processing 'RAMBU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 61.A. Jl. Raya Lemah Abang_RAMBU.geojson
Processing 'PJU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 61.A. Jl. Raya Lemah Abang_PJU.geojson
Processing 'RPPJ' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 61.A. Jl. Raya Lemah Abang_RPPJ.geojson
Processing 'PAGAR PENGAMAN' as Point geometry (regular sheet)
⚠️ Skipping 'PAGAR PENGAMAN' (No valid geometry found)
Processing 'MARKA' as MultiPoint (MARKA sheet)
⚠️ Skipping 'MARKA' (No valid geometry found)
Processing 'WARNING LIGHT' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 61.A. Jl. Raya Lemah Abang_WARNING LIGHT.geojson
Processing 'APILL' as Point geometry (regular sh

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 62.A. Akses Dryport Cikarang_RAMBU.geojson
Processing 'PJU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 62.A. Akses Dryport Cikarang_PJU.geojson
Processing 'RPPJ' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 62.A. Akses Dryport Cikarang_RPPJ.geojson
Processing 'PAGAR PENGAMAN' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Format Baru 62.A. Akses Dryport Cikarang_PAGAR PENGAMAN.geojson
Processing 'MARKA' as MultiPoint (MARKA sheet)
❌ Error processing Format Baru 62.A. Akses Dryport Cikarang.xlsx: None

[29/31] Processing: Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu).xlsx


c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)_RAMBU.geojson
Processing 'PJU' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)_PJU.geojson
Processing 'RPPJ' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)_RPPJ.geojson
Processing 'PAGAR PENGAMAN' as Point geometry (regular sheet)
⚠️ Skipping 'PAGAR PENGAMAN' (No valid geometry found)
Processing 'MARKA' as MultiPoint (MARKA sheet)
⚠️ Skipping 'MARKA' (No valid geometry found)
Processing 'WARNING LIGHT' as Point geometry (regular sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract GeoJSON\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)_WARNING LIGHT.geojson
Processing 'APILL' as Po

## 2. Application to Export Excel into Shapefile

### 2.1. Function Codes

In [22]:
import sys
import subprocess
import pkg_resources

def install_requirements(): #Install required packages if they are not already installed.
    required = {
        'pandas': '1.0.0',
        'openpyxl': '3.0.0',
        'geopandas': '0.9.0',
        'shapely': '1.7.0',
    }
    
    # Check what's installed
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    
    # Determine what needs to be installed
    missing = []
    update = []
    
    for package, min_version in required.items():
        if package not in installed:
            missing.append(package)
        elif pkg_resources.parse_version(installed[package]) < pkg_resources.parse_version(min_version):
            update.append(package)
    
    # If packages need to be installed or updated
    if missing or update:
        print("Some required packages are missing or need to be updated.")
        print(f"Missing: {', '.join(missing) if missing else 'None'}")
        print(f"Need update: {', '.join(update) if update else 'None'}")
        
        try:
            # Install missing packages
            if missing:
                print(f"Installing missing packages: {', '.join(missing)}")
                subprocess.check_call([sys.executable, "-m", "pip", "install"] + missing)
            
            # Update packages that need updating
            if update:
                print(f"Updating packages: {', '.join(update)}")
                subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade"] + update)
                
            print("All required packages have been installed/updated successfully!")
            
            # Re-import the modules to make them available
            if 'pandas' in missing or 'pandas' in update:
                global pd
                import pandas as pd
            if 'openpyxl' in missing or 'openpyxl' in update:
                global load_workbook
                from openpyxl import load_workbook
            if 'geopandas' in missing or 'geopandas' in update:
                global gpd
                import geopandas as gpd
            if 'shapely' in missing or 'shapely.geometry' in update:
                global Point, LineString
                from shapely.geometry import Point, LineString
                
        except Exception as e:
            print(f"Failed to install required packages: {str(e)}")
            print("Please manually install the required packages using:")
            print("pip install pandas openpyxl geopandas shapely")
            sys.exit(1)


if __name__ == "__main__": # Run the requirements check at the beginning
    install_requirements()

install_requirements()

import pandas as pd
from openpyxl import load_workbook
import geopandas as gpd
from shapely.geometry import Point, LineString
import json
import glob
import os
from pathlib import Path


def unique_column_names(columns): # Ensure column names are unique by appending suffix.
    seen = {}
    new_columns = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)
    return new_columns

def clean_column_names(columns): # Standardize column names by capitalizing each word properly.
    cleaned_columns = []
    seen = {}
    for col in columns:
        col = str(col).strip()
        
        # Remove columns with "Rekap" in the name
        if "rekap" in col.lower():
            continue
        
        # Fix for duplicated words (like "No No", "Detail Lokasi Detail Lokasi")
        words = col.split()
        if len(words) >= 2:
            # Check for repeated word patterns
            half_length = len(words) // 2
            if words[:half_length] == words[half_length:] and len(words) % 2 == 0:
                # If the first half matches the second half, only use the first half
                col = " ".join(words[:half_length])
        
        # Capitalize words
        col = " ".join(word.capitalize() for word in col.split())
        
        # Handle duplicates
        if col in seen:
            seen[col] += 1
            col = f"{col} {seen[col]}"
        else:
            seen[col] = 0
        
        # Shapefile has column name length limitation (10 chars)
        # We'll handle this by truncating to 10 chars while ensuring uniqueness
        col = col[:10]
        
        cleaned_columns.append(col)
    return cleaned_columns

def fix_coordinates(row, lat_col, lon_col): #Fix latitude and longitude values that may be in the wrong format.
    lat, lon = row[lat_col], row[lon_col]
    original_lat, original_lon = lat, lon  # Store original values for debugging
    
    # Handle string values with commas (e.g., "-69,694,951" or "1,065,663,308")
    if isinstance(lat, str):
        try:
            lat = float(lat.replace(',', ''))
        except (ValueError, AttributeError):
            lat = pd.NA
    
    if isinstance(lon, str):
        try:
            lon = float(lon.replace(',', ''))
        except (ValueError, AttributeError):
            lon = pd.NA
    
    # First attempt to detect the scale by number of digits
    if pd.notna(lat):
        lat_abs = abs(lat)
        # For values like -6448977 (which should be around -6.4 degrees)
        if 1_000_000 < lat_abs < 10_000_000 and str(int(lat_abs)).startswith(('6', '7', '8', '9')):
            lat = lat / 1_000_000
        elif lat_abs > 90:
            # General scaling rules based on magnitude
            if lat_abs > 10_000_000:  # Very large values
                lat = lat / 10_000_000
            elif lat_abs > 1_000_000:  # Large values (common for Indonesia coords)
                lat = lat / 1_000_000
            elif lat_abs > 90_000:
                lat = lat / 1_000
    
    if pd.notna(lon):
        lon_abs = abs(lon)
        # For Indonesian longitudes (usually around 106-110 degrees)
        if 100_000_000 < lon_abs < 1_500_000_000:
            lon = lon / 10_000_000
        elif lon_abs > 180:
            # General scaling rules based on magnitude
            if lon_abs > 10_000_000:
                lon = lon / 10_000_000
            elif lon_abs > 1_000_000:
                lon = lon / 1_000_000
            elif lon_abs > 180_000:
                lon = lon / 1_000
    
    # Final validation with better error reporting
    if pd.notna(lat) and (lat < -90 or lat > 90):
        print(f"Warning: Invalid latitude after correction: {lat} (original: {original_lat})")
        
        # Last attempt for specific problematic values
        if -10000 < lat < -90 or 90 < lat < 10000:
            lat = lat / 100
            print(f"  Attempting additional scaling: now {lat}")
        
        if lat < -90 or lat > 90:  # Still invalid
            lat = pd.NA
    
    if pd.notna(lon) and (lon < -180 or lon > 180):
        print(f"Warning: Invalid longitude after correction: {lon} (original: {original_lon})")
        
        # Last attempt for specific problematic values
        if -18000 < lon < -180 or 180 < lon < 18000:
            lon = lon / 100
            print(f"  Attempting additional scaling: now {lon}")
            
        if lon < -180 or lon > 180:  # Still invalid
            lon = pd.NA
        
    return pd.Series([lat, lon])


def save_shapefile(gdf, output_path): #Save GeoDataFrame to Shapefile format with warnings suppressed
    
    try:
        # Truncate column names to 10 characters ourselves to avoid warnings
        # This proactively handles the limitation rather than relying on geopandas/pyogrio to do it
        orig_columns = gdf.columns.tolist()
        truncated_columns = [str(col)[:10] for col in orig_columns]
        
        # Create a mapping dictionary to show what changed
        column_mapping = {orig: trunc for orig, trunc in zip(orig_columns, truncated_columns)}
        
        # Check for duplicates after truncation and make them unique
        seen = {}
        unique_truncated = []
        for col in truncated_columns:
            if col in seen:
                seen[col] += 1
                unique_truncated.append(f"{col[:7]}_{seen[col]}")
            else:
                seen[col] = 0
                unique_truncated.append(col)
        
        # Rename the columns with the unique truncated names
        gdf.columns = unique_truncated
        
        # Import warnings to suppress them
        import warnings
        
        # Suppress specific warnings
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning, message='.*truncated when saved to ESRI Shapefile.*')
            warnings.filterwarnings('ignore', category=RuntimeWarning, message='.*Normalized/laundered field name.*')
            
            # Save to shapefile with warnings suppressed
            gdf.to_file(output_path, driver="ESRI Shapefile")
        
        print(f"✅ Saved: {output_path}")
                
    except Exception as e:
        print(f"❌ Error saving shapefile {output_path}: {str(e)}")

def find_coordinate_columns(df, prefix, column_type): # Find coordinate columns with various naming patterns returns column name if found, None otherwise
    patterns = []
    
    if prefix == 'start':
        if column_type == 'lat':
            patterns = [
                'awal latitude', 'awal lat', 
                'koordinat latitude', 'koordinat lat',
                'latitude awal', 'lat awal',
                'latitude 1', 'lat 1'
            ]
        else:  # longitude
            patterns = [
                'awal longitude', 'awal lon', 
                'koordinat longitude', 'koordinat lon',
                'longitude awal', 'lon awal',
                'longitude 1', 'lon 1'
            ]
    else:  # end
        if column_type == 'lat':
            patterns = [
                'akhir latitude', 'akhir lat',
                'latitude akhir', 'lat akhir',
                'latitude 2', 'lat 2'
            ]
        else:  # longitude
            patterns = [
                'akhir longitude', 'akhir lon',
                'longitude akhir', 'lon akhir',
                'longitude 2', 'lon 2'
            ]
    
    # If we're looking for start coordinates, and just a single coordinate exists (no start/end distinction)
    if prefix == 'start' and column_type == 'lat':
        # Add cases where latitude exists without start/end specifier
        patterns.extend(['latitude', 'lat'])
    elif prefix == 'start' and column_type == 'lon':
        # Add cases where longitude exists without start/end specifier
        patterns.extend(['longitude', 'lon'])
    
    # Search for column containing any of the patterns
    for pattern in patterns:
        for col in df.columns:
            if isinstance(col, str) and pattern in col.lower():
                return col
    
    return None

def flatten_excel_to_shapefile(file_path, output_folder): #Convert all sheets from an Excel file to Shapefile, handling Point, MultiPoint and LineString geometries

    # Load workbook
    wb = load_workbook(file_path, data_only=True)

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Unmerge cells and fill values
        for merge in list(ws.merged_cells):
            ws.unmerge_cells(str(merge))
            top_left = ws.cell(merge.min_row, merge.min_col).value
            for row in range(merge.min_row, merge.max_row + 1):
                for col in range(merge.min_col, merge.max_col + 1):
                    ws.cell(row, col, top_left)

        # Convert to DataFrame
        data = list(ws.values)
        df = pd.DataFrame(data)

        # Check if DataFrame is empty or has too few rows
        if df.empty or len(df) < 3:
            print(f"⚠️ Skipping '{sheet_name}' (Empty sheet or insufficient data)")
            continue

        # Find rows containing "NO" (safer approach)
        try:
            header_indices = df[df.apply(lambda x: x.astype(str).str.contains("NO", case=False, na=False)).any(axis=1)].index
            if len(header_indices) == 0:
                print(f"⚠️ Skipping '{sheet_name}' (No header row with 'NO' found)")
                continue
            header_index = header_indices[0]
        except Exception as e:
            print(f"⚠️ Skipping '{sheet_name}' (Error finding header row: {str(e)})")
            continue

        # Use the identified header row
        df.columns = df.iloc[header_index].astype(str).str.strip()

        # Remove empty columns
        df = df.dropna(axis=1, how="all")

        # First pass of REKAP filtering - do this early and more aggressively
        try:
            rekap_mask = df.columns.astype(str).str.contains("REKAP", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} REKAP columns in first pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering REKAP columns - {str(e)}")

        # Drop rows above and including header, plus the empty row after header
        rows_to_drop = list(range(0, header_index + 2))
        if len(df) > max(rows_to_drop) + 1:  # Make sure we have enough rows
            df = df.drop(index=rows_to_drop).reset_index(drop=True)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (Not enough data rows after header)")
            continue

        # Improved header row merging
        if len(df) >= 2:
            # Convert all values to strings to handle None values safely
            first_row = df.iloc[0].astype(str).replace('None', '').replace('nan', '')
            second_row = df.iloc[1].astype(str).replace('None', '').replace('nan', '')
            
            # Smart merging to avoid duplication
            merged_header = []
            for a, b in zip(first_row, second_row):
                a = a.strip()
                b = b.strip()
                
                # Skip columns with "REKAP" in the name - additional check
                if "rekap" in a.lower() or "rekap" in b.lower():
                    merged_header.append("TO_BE_REMOVED")  # Mark for removal
                    continue
                
                if not a and not b:
                    merged_header.append("Column_" + str(len(merged_header)))
                elif not a:
                    merged_header.append(b)
                elif not b:
                    merged_header.append(a)
                else:
                    # Check if one is contained in the other to avoid redundancy
                    if a.lower() in b.lower():
                        merged_header.append(b)
                    elif b.lower() in a.lower():
                        merged_header.append(a)
                    else:
                        merged_header.append(f"{a} {b}")
            
            # Ensure column names are unique
            df.columns = unique_column_names(merged_header)
            
            # Remove any columns marked for removal
            df = df.loc[:, ~df.columns.str.contains("TO_BE_REMOVED")]
            
            # Remove the first two rows used for headers
            df = df.drop(index=[0, 1]).reset_index(drop=True)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (Not enough rows for headers)")
            continue

        # Second pass of REKAP filtering after merging headers
        try:
            rekap_mask = df.columns.astype(str).str.contains("REKAP", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} REKAP columns in second pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering REKAP columns (second pass) - {str(e)}")

        # Normalize column names for consistent detection (safely handle None values)
        df.columns = [str(col).lower().strip() if col is not None else f"col_{i}" 
                      for i, col in enumerate(df.columns)]
        
        # Third pass of REKAP filtering after normalizing
        try:
            rekap_mask = df.columns.str.contains("rekap", case=False, na=False)
            if rekap_mask.any():
                df = df.loc[:, ~rekap_mask]
                print(f"Removed {rekap_mask.sum()} rekap columns in third pass")
        except Exception as e:
            print(f"Warning in '{sheet_name}': Error filtering rekap columns (third pass) - {str(e)}")
        
        # Check if the sheet is MARKA or PAGAR PENGAMAN by name or column content
        is_marka_sheet = "marka" in sheet_name.lower() or any("marka" in col for col in df.columns)
        is_pagar_pengaman_sheet = "pagar pengaman" in sheet_name.lower() or any("pagar pengaman" in col for col in df.columns)

        # Find all coordinate columns using the improved function
        start_lat_col = find_coordinate_columns(df, 'start', 'lat')
        start_lon_col = find_coordinate_columns(df, 'start', 'lon')
        end_lat_col = find_coordinate_columns(df, 'end', 'lat')
        end_lon_col = find_coordinate_columns(df, 'end', 'lon')
        
        # Check available coordinate patterns
        has_start_coords = start_lat_col is not None and start_lon_col is not None
        has_end_coords = end_lat_col is not None and end_lon_col is not None

        if not has_start_coords:
            print(f"⚠️ Skipping '{sheet_name}' (No valid start coordinate columns found)")
            continue

        # Convert all coordinate columns to numeric
        if has_start_coords:
            df[start_lat_col] = pd.to_numeric(df[start_lat_col], errors='coerce')
            df[start_lon_col] = pd.to_numeric(df[start_lon_col], errors='coerce')
            
            # Fix coordinates if needed
            df[[start_lat_col, start_lon_col]] = df.apply(fix_coordinates, axis=1, lat_col=start_lat_col, lon_col=start_lon_col)
        
        if has_end_coords:
            df[end_lat_col] = pd.to_numeric(df[end_lat_col], errors='coerce')
            df[end_lon_col] = pd.to_numeric(df[end_lon_col], errors='coerce')
            
            # Fix coordinates if needed
            df[[end_lat_col, end_lon_col]] = df.apply(fix_coordinates, axis=1, lat_col=end_lat_col, lon_col=end_lon_col)
        
        # Check if the end coordinates actually contain valid data for LineString
        if has_end_coords:
            has_valid_end_coords = not df[end_lat_col].isna().all() and not df[end_lon_col].isna().all()
            
            # Check if we have at least one row with both coordinates valid
            valid_pairs = ((df[start_lat_col].notna() & df[start_lon_col].notna()) & 
                           (df[end_lat_col].notna() & df[end_lon_col].notna())).any()
            
            # Only use LineString if we have valid end coordinates and at least one pair is complete
            use_linestring = has_valid_end_coords and valid_pairs and is_pagar_pengaman_sheet
        else:
            use_linestring = False
        
        # Import MultiPoint if we need it for MARKA
        if is_marka_sheet and start_lat_col is not None and start_lon_col is not None:
            from shapely.geometry import MultiPoint
        
        # Determine geometry type based on sheet name and available coordinates
        if use_linestring:
            # LineString geometry with start/end points for PAGAR PENGAMAN
            print(f"Processing '{sheet_name}' as LineString (PAGAR PENGAMAN or similar)")
            
            # Create LineString geometry
            df["geometry"] = df.apply(
                lambda row: LineString([
                    (row[start_lon_col], row[start_lat_col]),
                    (row[end_lon_col], row[end_lat_col])
                ]) if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) and 
                     pd.notna(row[end_lon_col]) and pd.notna(row[end_lat_col]) else None,
                axis=1
            )
            
            # Exclude coordinate columns from properties
            exclude_cols = [start_lat_col, start_lon_col, end_lat_col, end_lon_col, "geometry"]
            
        elif is_marka_sheet:
            # MultiPoint geometry for MARKA
            print(f"Processing '{sheet_name}' as MultiPoint (MARKA sheet)")
            
            # Create MultiPoint geometry with both start and end coordinates if available
            if has_end_coords:
                df["geometry"] = df.apply(
                    lambda row: MultiPoint([
                        (row[start_lon_col], row[start_lat_col]),
                        (row[end_lon_col], row[end_lat_col])
                    ]) if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) and 
                         pd.notna(row[end_lon_col]) and pd.notna(row[end_lat_col]) else
                    # Fallback to single point if only start coordinates are valid
                    MultiPoint([(row[start_lon_col], row[start_lat_col])])
                    if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) else None,
                    axis=1
                )
                exclude_cols = [start_lat_col, start_lon_col, end_lat_col, end_lon_col, "geometry"]
            else:
                # Only start coordinates available for MultiPoint
                df["geometry"] = df.apply(
                    lambda row: MultiPoint([(row[start_lon_col], row[start_lat_col])])
                    if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) else None,
                    axis=1
                )
                exclude_cols = [start_lat_col, start_lon_col, "geometry"]
        else:
            # Default Point geometry for other sheets
            print(f"Processing '{sheet_name}' as Point geometry (standard sheet)")
            
            # Create Point geometry with start coordinates
            df["geometry"] = df.apply(
                lambda row: Point(row[start_lon_col], row[start_lat_col]) 
                if pd.notna(row[start_lon_col]) and pd.notna(row[start_lat_col]) else None,
                axis=1
            )
            
            # Exclude coordinate columns from properties
            exclude_cols = [start_lat_col, start_lon_col]
            if has_end_coords:
                exclude_cols.extend([end_lat_col, end_lon_col])
            exclude_cols.append("geometry")
        
        # Drop rows where geometry is None
        df = df.dropna(subset=["geometry"]).reset_index(drop=True)

        # Only create GeoDataFrame if there are valid geometries
        if len(df) > 0 and not df["geometry"].isnull().all():
            # Fourth pass of REKAP filtering
            # Filter REKAP columns before creating properties
            properties_cols = [col for col in df.columns if col not in exclude_cols]
            properties_cols = [col for col in properties_cols if 'rekap' not in str(col).lower()]
            
            gdf = gpd.GeoDataFrame(df[properties_cols + ["geometry"]], crs="EPSG:4326")
            
            # Apply column renaming after creating the GeoDataFrame
            gdf.columns = clean_column_names(gdf.columns)

            # Fifth pass REKAP filter - case insensitive filter after cleaning column names
            gdf = gdf.loc[:, ~gdf.columns.astype(str).str.contains("rekap", case=False, na=False)]
            
            # Remove unwanted "None_" and "None" columns
            gdf = gdf.loc[:, ~gdf.columns.astype(str).str.match(r"^None$|None_", na=False)]

            # Remove " None" from remaining column names
            gdf.columns = gdf.columns.astype(str).str.replace(r"\sNone\b", "", regex=True).str.strip()

            # Create output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Define output file path - changed from .geojson to .shp
            output_path = os.path.join(output_folder, f"{sheet_name}.shp")

            # One final check for REKAP columns before saving
            if any('rekap' in str(col).lower() for col in gdf.columns):
                print(f"Warning: Some REKAP columns still exist after all filtering: {[col for col in gdf.columns if 'rekap' in str(col).lower()]}")
                # Last resort: manually drop these columns
                gdf = gdf.loc[:, ~gdf.columns.astype(str).str.contains("rekap", case=False, na=False)]

            # Save the Shapefile using our new save_shapefile function
            save_shapefile(gdf, output_path)
        else:
            print(f"⚠️ Skipping '{sheet_name}' (No valid geometry found)")
            continue  # Skip processing this sheet if no valid geometries exist

def process_excel_folder(input_folder, output_base_folder): # Process all Excel files in a folder and convert them to Shapefiles
    
    # Create the 'Extract Shapefile' folder inside the output folder - changed from 'Extract GeoJSON'
    output_folder = os.path.join(output_base_folder, "Extract Shapefile")
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all Excel files in the input folder
    excel_extensions = ['*.xlsx', '*.xls', '*.xlsm']
    excel_files = []
    
    for ext in excel_extensions:
        excel_files.extend(glob.glob(os.path.join(input_folder, ext)))
    
    if not excel_files:
        print(f"⚠️ No Excel files found in {input_folder}")
        return
    
    # Process each Excel file
    for i, file_path in enumerate(excel_files, 1):
        file_name = os.path.basename(file_path)
        print(f"\n[{i}/{len(excel_files)}] Processing: {file_name}")
        
        # Create a subfolder for each Excel file
        file_output_folder = os.path.join(output_folder, Path(file_name).stem)
        os.makedirs(file_output_folder, exist_ok=True)
        
        try:
            flatten_excel_to_shapefile(file_path, file_output_folder)
            print(f"✓ Completed processing: {file_name}")
        except Exception as e:
            print(f"❌ Error processing {file_name}: {str(e)}")
    
    print(f"\n🎉 All Excel files processed. Output saved to: {output_folder}")

### 2.2. Run Function Excel to Shapefile

In [24]:
input_folder = r"C:\Users\kanzi\Documents\Part Time Job\Data Hasil Survey1"  # Fill with the path file of excel
output_base_folder = r"C:\Users\kanzi\Documents\Part Time Job\Hasil Export"  # Fill with the path folder of export result
process_excel_folder(input_folder, output_base_folder) # Run the function!


[1/29] Processing: 01. Cileungsi - Cibeet.xlsx


c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - Cibeet\RAMBU.shp
Processing 'PJU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - Cibeet\PJU.shp
Processing 'RPPJ' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - Cibeet\RPPJ.shp
Processing 'PAGAR PENGAMAN' as LineString (PAGAR PENGAMAN or similar)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - Cibeet\PAGAR PENGAMAN.shp
Processing 'MARKA' as MultiPoint (MARKA sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - Cibeet\MARKA.shp
Processing 'WARNING LIGHT' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\01. Cileungsi - C

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\02. CILEUNGSI - CIBINONG (CITEUREUP)\RAMBU.shp
Processing 'PJU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\02. CILEUNGSI - CIBINONG (CITEUREUP)\PJU.shp
Processing 'RPPJ' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\02. CILEUNGSI - CIBINONG (CITEUREUP)\RPPJ.shp
Processing 'PAGAR PENGAMAN' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\02. CILEUNGSI - CIBINONG (CITEUREUP)\PAGAR PENGAMAN.shp
Processing 'MARKA' as MultiPoint (MARKA sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\02. CILEUNGSI - CIBINONG (CITEUREUP)\MARKA.shp
Processing 'WARNING LIGHT' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents

...

Processing 'RAMBU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 61.A. Jl. Raya Lemah Abang\RAMBU.shp
Processing 'PJU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 61.A. Jl. Raya Lemah Abang\PJU.shp
Processing 'RPPJ' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 61.A. Jl. Raya Lemah Abang\RPPJ.shp
Processing 'PAGAR PENGAMAN' as Point geometry (standard sheet)
⚠️ Skipping 'PAGAR PENGAMAN' (No valid geometry found)
Processing 'MARKA' as MultiPoint (MARKA sheet)
⚠️ Skipping 'MARKA' (No valid geometry found)
Processing 'WARNING LIGHT' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 61.A. Jl. Raya Lemah Abang\WARNING LIGHT.shp
Processing 'APILL' as Point geometry (standard shee

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 62.A. Akses Dryport Cikarang\RAMBU.shp
Processing 'PJU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 62.A. Akses Dryport Cikarang\PJU.shp
Processing 'RPPJ' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 62.A. Akses Dryport Cikarang\RPPJ.shp
Processing 'PAGAR PENGAMAN' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 62.A. Akses Dryport Cikarang\PAGAR PENGAMAN.shp
Processing 'MARKA' as MultiPoint (MARKA sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Format Baru 62.A. Akses Dryport Cikarang\MARKA.shp
Processing 'WARNING LIGHT' as Point geometry (standard sheet)
⚠️ Skipping '

c:\Users\kanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Processing 'RAMBU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)\RAMBU.shp
Processing 'PJU' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)\PJU.shp
Processing 'RPPJ' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)\RPPJ.shp
Processing 'PAGAR PENGAMAN' as Point geometry (standard sheet)
⚠️ Skipping 'PAGAR PENGAMAN' (No valid geometry found)
Processing 'MARKA' as MultiPoint (MARKA sheet)
⚠️ Skipping 'MARKA' (No valid geometry found)
Processing 'WARNING LIGHT' as Point geometry (standard sheet)
✅ Saved: C:\Users\kanzi\Documents\Part Time Job\Hasil Export\Extract Shapefile\Ludira 27.A Jl. Bhayangkara (Pelabuhan Ratu)\WARNING LIGHT.shp
Processing 'APILL' as Point